## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import random
import matplotlib.pyplot as plt
import scipy.stats as stss
import numpy as np
import timeit
import requests
import json
from citipy import citipy
import time
from datetime import datetime



# Import API key
from config import weather_api_key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

ModuleNotFoundError: No module named 'config'

In [ ]:
#Import CSV file

#data_load = "WeatherPy_Database.csv"

#city_weather_df = pd.read_csv(data_load)



In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()
print(city_data_df)

FileNotFoundError: [Errno 2] No such file or directory: 'Weather_Database/WeatherPy_database.csv'

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

max_temp = float(input("Enter maximum vacation temperature in °F:  "))
min_temp = float(input("Enter minnimum vacation temperature in °F:  "))


print("Your tollerable vacation temperature is: " + str(max_temp) + '°F to ' + str(min_temp) + '°F')

# ref  90 and 75



Enter maximum vacation temperature in °F:  90
Enter minnimum vacation temperature in °F:  75
Your tollerable vacation temperature is: 90.0°F to 75.0°F


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


NameError: name 'city_data_df' is not defined

In [ ]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.info()


In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name
hotel_df["Current Description"] = ""
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
hotel_df.describe

In [ ]:
# Set the parameters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}



# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()






# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    


    
    
        

In [ ]:
hotel_df

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df = hotel_df.drop('Current Description', axis=1)
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
print(clean_hotel_df)

In [ ]:
clean_hotel_df.info()

In [ ]:

#new_column_order = ["City", "Country", "Max Temp", "Lat", "Lng", "Hotel Name"]
#clean_hotel_df = clean_hotel_df[new_column_order]
clean_hotel_df.head(20)

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:

# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [ ]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
print(hotel_info)

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
    
f

In [ ]:
# 11b. Display the figure
fig